In [1]:
import itertools
import pandas as pd
from itertools import combinations
from datetime import date

from data_manager import DataManager
import analyze
from models import Player

dm = DataManager()

class Prop:
    def __init__(self, name, team, stat, threshold, odds, bet_type):
        self.name = name
        self.team = team
        self.stat = stat
        self.n = threshold
        self.odds = odds
        self.bet_type = bet_type
        self.PROB = self.get_prop_PROB()
        self.ev, self.house_prob = self.get_ev_and_implied_prob()
        self.print_out = f"""
            PLAYER: {self.name}
              STAT: {self.stat}
            THRESH: {self.n}
              ODDS: {self.odds}
              TYPE: {self.bet_type}
              PROB: {self.PROB}
                EV: {self.ev}
        HOUSE_PROB: {self.house_prob}
            """
        print(self.print_out)
        self.entry = {
            "PLAYER": self.name,
              "TEAM": self.team,
              "STAT": self.stat,
            "THRESH": self.n,
              "ODDS": self.odds,
              "TYPE": self.bet_type,
              "PROB": self.probability,
                "EV": self.ev,
        "HOUSE_PROB": self.house_prob
        }
    
    def get_prop_probability(self, last_n_games=25):
        player_id = dm.get_player_id(self.name)
        data = dm.get_and_save_player_data(player_id, self.name).sort_values(by='date', ascending=False).head(last_n_games).copy()
        # print(data.head())
        if self.bet_type == "over":
            return analyze.estimate_probability_poisson_over(data, self.stat, self.n)
        elif self.bet_type == "under":
            return analyze.estimate_probability_poisson_under(data, self.stat, self.n)
        else:
            raise ValueError("Invalid bet type. Use 'over' or 'under'.")
        
    def get_ev_and_implied_prob(self):
        odds = self.american_to_decimal(self.odds)
        house_probability = analyze.estimate_implied_probability(odds)
        ev = analyze.calculate_ev(self.probability, odds, 5)
        return ev, house_probability
    
    @staticmethod
    def american_to_decimal(american_odds):
        """Convert American odds to decimal odds."""
        if american_odds > 0:
            return 1 + (american_odds / 100)
        else:
            return 1 + (100 / abs(american_odds))
    

def extract_raw_data(file_path): # .csv
    # gets input from A1
    # Sample input text (use the content of your file here)
    raw_input = pd.read_csv(file_path)
    list_of_raw_input = list(raw_input.iloc[:, 0])
    return list_of_raw_input


def load_available_props():
    
    raw_input = extract_raw_data("prop_lines/prop_lines.csv")
    print("Received raw input.")
    stat_names = {
             'PointsSGP': "points",
            'AssistsSGP': "assists",
        'Threes MadeSGP': "fg3m",
           'ReboundsSGP': "rebounds",
   'Field Goals MadeSGP': "fgm",
             'StealsSGP': "steals",
             'BlocksSGP': "blocks",
        }
    #debug stat_name_inputs = extract_raw_data("prop_lines/player_prop_categories.csv")
    players = dm.query_players()
    player_names = [player.name for player in players]
    teams = dm.query_teams()
    team_names = [team.nickname for team in teams]
    row_of_interest = 0
    current_category = None
    current_player = None
    current_team = None
    records = []
    for _, item in enumerate(raw_input):
        if item is None: 
            continue
        if item in stat_names:
            current_category = stat_names[item]
            print(f"Loading {current_category} props.")
        if item != current_team:
            if item in team_names:
                current_team = item
        if item in player_names:
            current_player = item 
            assert current_team
            record = [current_player, current_team, current_category]
            row_of_interest = 6


        if row_of_interest:
            row_of_interest -= 1
            if row_of_interest < 5:
                record.append(item)
                if row_of_interest == 1:
                    records.append(record)
                    record = []
    df = pd.DataFrame.from_records(records, columns=["player_name", "team", "stat", "over_threshold", "over_odds", "under_threshold", "under_odds"])
    
    df['player_name'] = df['player_name'].astype(str)
    df['team'] = df['team'].astype(str)
    df['stat'] = df['stat'].astype(str)
    df['over_threshold'] = df['over_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['under_threshold'] = df['under_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['over_odds'] = df['over_odds'].astype(int)
    df['under_odds'] = df['under_odds'].astype(int)

    return df


def get_analyzed_props(available_props):
    props = []
    for _, row in available_props.iterrows():
        print(row)
        for bet_type in ["over", "under"]:
            prop = Prop(
                    name=row["player_name"], 
                    team=row["team"],
                    stat=row["stat"], 
                threshold=row[f"{bet_type}_threshold"], 
                    odds=row[f"{bet_type}_odds"], 
                bet_type=bet_type
                )
            props.append(prop)    
            print("Prop object created.")

    return props


def generate_heterogenous_combinations(df, n):

    # Generate all combinations of n rows
    combinations = list(itertools.combinations(df.index, n))

    # Function to evaluate heterogeneity of a combination
    def evaluate_heterogeneity(comb, df):
        comb_list = list(comb)
        players = df.loc[comb_list, 'PLAYER']
        stats = df.loc[comb_list, 'STAT']
        teams = df.loc[comb_list, 'TEAM']
        # Calculate a simple heterogeneity score (you can define your own logic)
        player_score = len(set(players))
        stat_score = len(set(stats))
        team_score = len(set(teams))
        return player_score + stat_score + team_score

    # Evaluate all combinations and sort them by heterogeneity score
    comb_scores = [(comb, evaluate_heterogeneity(comb, df)) for comb in combinations]
    comb_scores_sorted = sorted(comb_scores, key=lambda x: x[1], reverse=True)

    # Select the most heterogeneous combinations (you can define how many you want)
    top_combinations = comb_scores_sorted # Top 5 combinations for example

    # Display the most heterogeneous combinations
    for comb, score in top_combinations:
        print(f"Combination: {comb}, Score: {score}")
        print(df.loc[list(comb)])
        print()

    # Optional: Convert combinations to DataFrame
    top_comb_dfs = [(df.loc[list(comb)], score) for comb, score in top_combinations]

    return top_comb_dfs


def generate_unique_combinations(df, combo_length):
    all_combinations = list(combinations(df.index, combo_length))
    
    unique_combinations = set()
    valid_combinations = []
    
    for combo in all_combinations:
        players_in_combo = df.loc[list(combo), 'Player']
        if players_in_combo.duplicated().sum() == 0:
            sorted_combo = tuple(sorted(df.loc[list(combo)].apply(lambda row: (row['Player'], row['Category'], row['Bet']), axis=1)))
            if sorted_combo not in unique_combinations:
                unique_combinations.add(sorted_combo)
                valid_combinations.append(combo)
    
    return valid_combinations

available_props = load_available_props()
analyzed_props = get_analyzed_props(available_props)
print(analyzed_props)
filter_players = ['Naz Reid', 'Rudy Gobert', "Nickeil Alexander-Walker"]
filtered_df = dm.filter_props(analyzed_props, filter_players)



Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.
player_name         Rudy Gobert
team               Timberwolves
stat                     points
over_threshold             10.5
over_odds                  -105
under_threshold            10.5
under_odds                 -130
Name: 0, dtype: object

            PLAYER: Rudy Gobert
              STAT: points
            THRESH: 10.5
              ODDS: -105
              TYPE: over
              PROB: 0.8021287476034127
                EV: 2.8303044408904565
        HOUSE_PROB: 0.5121951219512195
            
Prop object created.

            PLAYER: Rudy Gobert
              STAT: points
            THRESH: 10.5
              ODDS: -130
              TYPE: under
              PROB: 0.19787125239658732
                EV: -3.2496004595686507
        HOUSE_PROB: 0.5652173913043479
            
Prop object created.
player

In [39]:
import random


def distribute_props_into_parlays(props_df, n_parlays):
    """
    Distributes props into a specified number of parlays based on their probabilities.

    :param props_df: pandas DataFrame with columns 'player', 'team', 'threshold', 'odds', 'bet_type', 'probability'
    :param n_parlays: Number of parlays to distribute the props into
    :return: List of parlays, where each parlay is a list of props
    """
    # Initialize empty parlays
    parlays = [[] for _ in range(n_parlays)]

    # Calculate total probability
    total_probability = props_df["PROB"].sum()

    # Distribute props into parlays based on their probabilities
    for index, row in props_df.iterrows():
        # Determine how many parlays to distribute this prop into
        num_parlays_for_prop = int(round((row["PROB"] / total_probability) * n_parlays))
        num_parlays_for_prop = max(1, num_parlays_for_prop)  # Ensure at least 1 parlay

        # Randomly select parlays for this prop
        selected_parlays = random.sample(parlays, num_parlays_for_prop)
        for parlay in selected_parlays:
            parlay.append(row.to_dict())

    return parlays


def analyze_parlays(parlays):
    result = []

    for i, parlay in enumerate(parlays):
        probabilities = [prop['PROB'] for prop in parlay]
        odds = [prop['ODDS'] for prop in parlay]
        combined_ev = analyze.analyze_parlay(probabilities, odds, 5)
        combined_prob = analyze.calculate_combined_probability(probabilities)

        for prop in parlay:
            prop['PARLAY_ID'] = i
            prop['PARLAY_PROB'] = combined_prob
            prop['PARLAY_EV'] = combined_ev
        parlay_df = pd.DataFrame(parlay)
        result.append(parlay_df)
    
    # Concatenate all individual DataFrames
    parlays_df = pd.concat(result, ignore_index=True)
    return parlays_df


parlays = distribute_props_into_parlays(filtered_df, 5)
average_len_of_parlays = sum(map(len, parlays)) / len(parlays) if parlays else 0
print(f"AVERAGE LENGTH OF PARLAY: {average_len_of_parlays}")
parlays = analyze_parlays(parlays)
print(parlays)
parlays.to_csv("data_pile/parlays_df.csv")

AVERAGE LENGTH OF PARLAY: 7.2
Combined Probability: 0.0802
Combined Odds: 23.76
Expected Value (EV): $4.53
Combined Probability: 0.0012
Combined Odds: 3370.29
Expected Value (EV): $15.02
Combined Probability: 0.0107
Combined Odds: 593.55
Expected Value (EV): $26.70
Combined Probability: 0.2713
Combined Odds: 5.19
Expected Value (EV): $2.03
Combined Probability: 0.0146
Combined Odds: 145.09
Expected Value (EV): $5.61
                      PLAYER          TEAM      STAT  THRESH  ODDS   TYPE  \
0                Mike Conley  Timberwolves       fgm     3.5  -125   over   
1               Nikola Jokic       Nuggets   assists     8.5  -105  under   
2         Karl-Anthony Towns  Timberwolves    blocks     0.5   160   over   
3               Jamal Murray       Nuggets    blocks     0.5   160   over   
4               Nikola Jokic       Nuggets    points    30.5  -125  under   
5   Kentavious Caldwell-Pope       Nuggets    steals     1.5  -200  under   
6               Nikola Jokic       Nugget

In [43]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.utils.dataframe import dataframe_to_rows

def highlight_rows_with_color_scale(df, column_letter, output_file):
    """
    Highlight entire rows in a DataFrame based on the value in a specific column with a color scale and save to an Excel file.

    :param df: pandas DataFrame
    :param column_letter: Column letter to base the condition on (e.g., 'K')
    :param output_file: Output Excel file name
    """
    # Create a workbook and select the active worksheet
    wb = Workbook()
    ws = wb.active

    # Add the DataFrame to the worksheet
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Define the range for the conditional formatting
    range_string = f"{column_letter}2:{column_letter}{ws.max_row}"

    # Create a color scale rule
    color_scale_rule = ColorScaleRule(
        start_type='min', start_color='FF0000',  # Red
        mid_type='percentile', mid_value=50, mid_color='FFFF00',  # Yellow
        end_type='max', end_color='00FF00'  # Green
    )

    # Apply the rule to each row
    for row in range(2, ws.max_row + 1):
        cell_ref = f"{column_letter}{row}"
        ws.conditional_formatting.add(f"A{row}:{chr(64 + ws.max_column)}{row}", color_scale_rule)

    # Save the workbook
    wb.save(output_file)

# Example usage
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9],
    'D': [10, 11, 12],
    'E': [13, 14, 15],
    'F': [16, 17, 18],
    'G': [19, 20, 21],
    'H': [22, 23, 24],
    'I': [25, 26, 27],
    'J': [28, 29, 30],
    'K': [4, 6, 9]
}
df = pd.DataFrame(data)

# Apply the highlighting based on values in column 'K'
highlight_rows_with_color_scale(df, 'K', 'highlighted_rows_color_scale.xlsx')
